# 기본세팅

In [1]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
The following packages were automatically installed and are no longer required:
  glib-networking-common glib-networking-services gstreamer1.0-x libaa1
  libavc1394-0 libcolord2 libdconf1 libdv4 libepoxy0 libgudev-1.0-0
  libiec61883-0 libproxy1v5 libraw1394-11 libshout3 libsoup2.4-common
  libtag1v5 libtag1v5-vanilla libu2f-udev libwavpack1 udev
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/X11
	/usr/share/fonts/cMap
	/usr/share/fonts/cmap
	/usr/share/fonts/opentype
	/usr/share/fonts/truetype
	/usr/share/fonts/type1
	/usr/share/fonts/X11/Type1
	/usr/share/fonts/X11/encodings
	/usr/share/fonts/X11/misc
	/usr/share/fonts/X11/util
	/usr/share/fonts/cmap/adobe-cns1
	/us

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
import scipy.stats as stats

# 서비스 방향 모색을 위한 분석
## 분석 목적 : 설정된 페르소나에 맞춤형 프로그램 제공을 위한 서비스 방향 모색
- 페르소나 : 희귀 신경계 질환을 앓고 있으며, 취사/청소 능력이 없는 서울 소재의 70대 독거 남성 

### 분석 방향
1) 신경계통 질환 환자들의 니즈 파악

#### 전체 희귀질환 환자들의 니즈 파악

##### dataset 불러오기
1. 신경계 희귀질환 커뮤니티
   - 한국다발성경화증협회
   - 네이버카페 검색
   - MASTERING MS 다발성경화증 '행복한 동행'
   - 길랑바레, 밀러피셔 증후군
   - 파킨슨병
   - 네이버카페 병명 검색

In [4]:
# mongo에서 db 불러오는 함수
import pymongo
def dbconnect(collection) :
    client=pymongo.MongoClient('mongodb://trainings.iptime.org',45003)
    db= client['data_analysis']
    collection = db[collection]
    data = collection.find()
    df = pd.DataFrame(data)
    return df

In [5]:
# 중복값 제거하는 함수(contents를 기준으로)
def drop_duplicates(df, contents) : 
    df.drop_duplicates(subset=contents, keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [6]:
# 특정 열에서 list를 str으로 변환하는 함수
def list_join(row) :
    row = ' '.join(row)
    return row

In [7]:
# 특정 열에서 \n 삭제하는 함수
def remove_text(row) :
    row = row.replace('\n',' ')
    return row

##### 데이터 전처리

###### KMSS

In [8]:
df_KMSS = dbconnect("KMSS")
df_KMSS

,_id,title,view,date,contents,reply
0,664162e6b6946f97c6276ef7,늦었습니다.,15회,2024.05.11 19:36:46,어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.\n오늘 아침에 퇴원을 하셔서...,이현호\n2024.05.13 09:17:41\n조인구님 안녕하세요.\n조인구님 글이...
1,664162ecb6946f97c6276ef8,Only Love - Nana Mouskouri: with Lyrics,14회,2024.05.07 16:13:59,상대에게 신뢰받는 5가지 방법 \n\n 1. 대화할 때 혼자만 이야기하지 말라....,이현호\n2024.05.08 10:25:51\n세상 살아가면서 대화가 잘 통하고 내...
2,664162f3b6946f97c6276ef9,하비(Harvey),18회,2024.05.03 16:19:36,https://youtu.be/q_TJlMUHfp4\n하비(Harvey) - 1부\...,구애경\n2024.05.03 19:05:44\nhttps://m.blog.naver...
3,664162f9b6946f97c6276efa,4월아 안녕~~.,20회,2024.04.30 15:06:42,"4월이 마침표를 찍는 날입니다.\n날씨가 변화무쌍했습니다.(추웠다 더웠다, 비오고 ...",서영임\n2024.04.30 15:33:46\n같은 아픔을 나누는 우리 모임에 모일...
4,66416300b6946f97c6276efb,4월을 마무리 짓는 날입니다.,16회,2024.04.30 11:12:18,오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정의 달이라고 하는 5월입니다.\...,이현호\n2024.04.30 11:56:18\n어머니가 편찮으셔서 걱정이 많이 되시...
...,...,...,...,...,...,...
884,664179b8b6946f97c627726b,건강상태 알려주는 신체의 아침신호,139회,2019.07.26 14:27:44,건강상태 알려주는 신체의 아침신호\n \n깨어나는 아침에 나타나는 증상들을 주의 깊...,
885,664179bfb6946f97c627726c,[제안] 국립 희귀질환센터 설립,190회,2019.07.22 10:33:20,국립 희귀질환센터 설립\n 제안이유\n 사람의 자유와 행복은 건강한 신체에 있습...,
886,664179c5b6946f97c627726d,치아관리 13계명,141회,2019.06.22 00:30:45,치아관리 13계명\n \n \n 건강한 이야말로 무덤까지 함께 갈 평생 건강의 필...,
887,664179cbb6946f97c627726e,고혈압에 좋은 음식과 차와 운동입니다.,159회,2019.06.15 20:14:53,"일차성고혈압에 대해 좋은 (음식, 차, 운동)\n 일차성고혈압이란 혈액이 혈관 벽에...",


In [9]:
# 중복데이터 확인
df_KMSS['contents'].value_counts()
# 중복데이터 없음

contents
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
df_KMSS.reset_index(drop=True, inplace=True)
df_KMSS

,_id,title,view,date,contents,reply
0,664162e6b6946f97c6276ef7,늦었습니다.,15회,2024.05.11 19:36:46,어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.\n오늘 아침에 퇴원을 하셔서...,이현호\n2024.05.13 09:17:41\n조인구님 안녕하세요.\n조인구님 글이...
1,664162ecb6946f97c6276ef8,Only Love - Nana Mouskouri: with Lyrics,14회,2024.05.07 16:13:59,상대에게 신뢰받는 5가지 방법 \n\n 1. 대화할 때 혼자만 이야기하지 말라....,이현호\n2024.05.08 10:25:51\n세상 살아가면서 대화가 잘 통하고 내...
2,664162f3b6946f97c6276ef9,하비(Harvey),18회,2024.05.03 16:19:36,https://youtu.be/q_TJlMUHfp4\n하비(Harvey) - 1부\...,구애경\n2024.05.03 19:05:44\nhttps://m.blog.naver...
3,664162f9b6946f97c6276efa,4월아 안녕~~.,20회,2024.04.30 15:06:42,"4월이 마침표를 찍는 날입니다.\n날씨가 변화무쌍했습니다.(추웠다 더웠다, 비오고 ...",서영임\n2024.04.30 15:33:46\n같은 아픔을 나누는 우리 모임에 모일...
4,66416300b6946f97c6276efb,4월을 마무리 짓는 날입니다.,16회,2024.04.30 11:12:18,오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정의 달이라고 하는 5월입니다.\...,이현호\n2024.04.30 11:56:18\n어머니가 편찮으셔서 걱정이 많이 되시...
...,...,...,...,...,...,...
884,664179b8b6946f97c627726b,건강상태 알려주는 신체의 아침신호,139회,2019.07.26 14:27:44,건강상태 알려주는 신체의 아침신호\n \n깨어나는 아침에 나타나는 증상들을 주의 깊...,
885,664179bfb6946f97c627726c,[제안] 국립 희귀질환센터 설립,190회,2019.07.22 10:33:20,국립 희귀질환센터 설립\n 제안이유\n 사람의 자유와 행복은 건강한 신체에 있습...,
886,664179c5b6946f97c627726d,치아관리 13계명,141회,2019.06.22 00:30:45,치아관리 13계명\n \n \n 건강한 이야말로 무덤까지 함께 갈 평생 건강의 필...,
887,664179cbb6946f97c627726e,고혈압에 좋은 음식과 차와 운동입니다.,159회,2019.06.15 20:14:53,"일차성고혈압에 대해 좋은 (음식, 차, 운동)\n 일차성고혈압이란 혈액이 혈관 벽에...",


In [11]:
# 각 컬럼별 전처리(제목)
df_KMSS['title']

0                                       늦었습니다.
1      Only Love - Nana Mouskouri: with Lyrics
2                                   하비(Harvey)
3                                    4월아 안녕~~.
4                             4월을 마무리 짓는 날입니다.
                        ...                   
884                         건강상태 알려주는 신체의 아침신호
885                          [제안] 국립 희귀질환센터 설립
886                                  치아관리 13계명
887                      고혈압에 좋은 음식과 차와 운동입니다.
888                  커피가 하루동안 인체에 미치는 영향이 무얼까?
Name: title, Length: 889, dtype: object

In [12]:
# 각 컬럼별 전처리(내용)
# 특정 열에서 \n 삭제
df_KMSS['contents'] = df_KMSS['contents'].str.replace('\n', '')

df_KMSS['contents']

0      어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원을 하셔서 집...
1       상대에게 신뢰받는 5가지 방법   1. 대화할 때 혼자만 이야기하지 말라.    ...
2      https://youtu.be/q_TJlMUHfp4하비(Harvey) - 1부htt...
3      4월이 마침표를 찍는 날입니다.날씨가 변화무쌍했습니다.(추웠다 더웠다, 비오고 흐린...
4      오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정의 달이라고 하는 5월입니다.우...
                             ...                        
884    건강상태 알려주는 신체의 아침신호 깨어나는 아침에 나타나는 증상들을 주의 깊게 살핀...
885     국립 희귀질환센터 설립  제안이유 사람의 자유와 행복은 건강한 신체에 있습니다. ...
886    치아관리 13계명    건강한 이야말로 무덤까지 함께 갈 평생 건강의 필수 동반자다...
887    일차성고혈압에 대해 좋은 (음식, 차, 운동) 일차성고혈압이란 혈액이 혈관 벽에 가...
888    대개의 사람들이 커피를 좋아한다.2015년에 시행한 여론조사에 따르면, 미국인 중 ...
Name: contents, Length: 889, dtype: object

In [13]:
# 각 컬럼별 전처리(댓글)
# 특정 열에서 \n 삭제
df_KMSS['reply'] = df_KMSS['reply'].str.replace('\n', '')
# 날짜와 고유명사 삭제
df_KMSS['reply'] = df_KMSS['reply'].str.replace(r'\b\w+\d{4}\.\d{2}\.\d{2}\s+\d{2}:\d{2}:\d{2}', '', regex=True)

df_KMSS['reply']

0      조인구님 안녕하세요.조인구님 글이 올라오지않아서 어머님의 건강을 걱정하고 있었습니다...
1      세상 살아가면서 대화가 잘 통하고 내 이야기 잘 경청해주고 공감해주는 사람이 제일 ...
2      https://m.blog.naver.com/PostView.naver?blogId...
3      같은 아픔을 나누는 우리 모임에 모일 수 없으면 찾아가야죠. 시간이 허락되면 동참하...
4      어머니가 편찮으셔서 걱정이 많이 되시겠어요.병원에 꽤 오래 계시네요.하루속히 쾌차하...
                             ...                        
884                                                     
885                                                     
886                                                     
887                                                     
888        흥미로운 글이네요💇잘 읽었습니다🙋✌커피 좋아하시는분들은 꼭 읽어봤으면 좋겠어요:)
Name: reply, Length: 889, dtype: object

In [14]:
KMSS_series = df_KMSS['title'] + ' ' + df_KMSS['contents']
KMSS_series.isna().sum()

0

In [15]:
# title, contents, reply 컬럼을 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
KMSS_series = df_KMSS['title'] + ' ' + df_KMSS['contents']
KMSS_list = KMSS_series.tolist()
KMSS_list

["늦었습니다. 어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원을 하셔서 집에 와서 여러가지 일을 하다보니 글 올리는 것이 생각이 났습니다.;오늘은 우리가 많이 먹는 포도에 대한 효능과 부작용에 대한 글입니다.포도 효능 11가지한여름이 지나가고 가을이 올 때쯤 되면 한참 달콤한 과일을 먹을 수 있어서 좋은 거 같다. 달콤한 복숭아도 좋고, 새콤한 자두도 좋지만 한 알 한 알 톡톡 터지는 식감이 좋은 포도도 너무 좋아해요. 아이들 간식부터 어르신들 입맛까지 사로잡은 과일로 누구나 좋아하는 음식으로 손꼽힌다.7~8월이 제철이어서 무더운 여름에 맛있게 먹을 수 있는 과일입니다. 특유의 향이 좋고 과즙이 풍부해서 과일 자체로도 맛있지만 주스로 마시거나 건포도로 가공해서 먹기도 한다. 우리나라는 이 과일을 꽤 옛날부터 먹어왔습니다.포도 효능1. 혈압을 조절해 준다.포도는 심장 건강에 도움이 되는 한 부분으로 혈압에 긍정적인 영향을 미칩니다. 혈압은 동맥혈관 벽에 대항한 혈액의 압력을 말한다. 이 혈액의 압력이 여러 원인으로 인해 높아진 상태를 고혈압이라 하는데, 우리나라 성인 인구의 30%가량이 고혈압 증상이 있는 것으로 추정하고 있습니다. 포도에는 칼륨과 섬유질이 풍부해서 나트륨의 배출을 돕습니다. 나트륨이 고혈압을 유발하는 원인 중 한 가지입니다. 포도를 적정량 꾸준히 섭취하다 보면 나트륨으로 인한 고혈압의 위험을 낮출 수 있습니다.2. 치매를 예방해 준다.포도 씨의 폴리페놀 성분은 알츠하이머의 진전을 늦춰줍니다. 연구에 의하면 실험용 생쥐에게 포도 씨의 폴리페놀 추출물을 5개월 동안 투여한 결과, 기억력 감퇴에 영향을 주는 ‘아밀로이드 베타-56’이라 하는 신경독소 물질이 감소한 것으로 나타났습니다.3. 항암 효과를 볼 수 있다.포도에는 폴리페놀이라는 항산화 성분이 포함되어 있습니다. 이들은 산화방지제 역할을 하여 암 관리와과 세포 손상 및 보호에 도움을 줄 수 있습니다. 특히 적포도에는 레스베라트롤이라는 폴리페놀의 일종인 성분이 많이 함유

In [64]:
df_behcet = dbconnect("persona1_behcet_freeboard")
df_behcet

,_id,contents_title,contents_writer,contents_date,click_count,contents_text,reply_list
0,6631f76c5bc2b816b6c98c6c,어느봄날에...,배영반,2024.04.07,19,한참 몸이 않좋아 열심히 치료하다\n잠시 한숨을 돌립니다.\n\n요즘 집압 공원에...,"[이재강 2024.04.10\n답변\n오~~뚱뚱해 보여요,, 각도 비스듬이 해서 찍..."
1,6631f7835bc2b816b6c98c6d,세상에서 가장 좋은 이별이란,한현옥,2024.03.19,19,폐암말기로 투병하던 길똥씨..\n방사선을 머리에 쐬여 영혼마저 암세포와 함께 파쇄됐...,[이재강 2024.03.19\n답변\n잘 읽었습니다.]
2,6631f7855bc2b816b6c98c6e,사람살려,한현옥,2024.03.01,22,00:00\n 00:00\n \n옻순을 얼마나 좋아하...,[]
3,6631f7875bc2b816b6c98c6f,봄이 왔어요,한현옥,2024.02.27,16,봄에 심을 햇강낭콩을 종류대로 사려고 오일장만 서면 가서 눈여겨 보았었다.\n작년....,[이재강 2024.02.27\n답변\n^^ 몸 생각하셔서 쉬엄쉬엄 농사 지으세요]
4,6631f7875bc2b816b6c98c70,신년맞이 떡국떡 감사의 글,이재강,2024.02.01,13,할리데이비슨 할리천사와 코리아챕터 감사드립니다~\n\n선정해주신 환우회 임원님들과 ...,[]
...,...,...,...,...,...,...,...
38573,6632c1545bc2b816b6ca2319,사랑전선에 또 문제가...,이은희,2005.04.12,82,참 공감가네요!저두 두달전에...헤어졌답니다!첨 만날때부터 걱정은 됐지만...괜찮을...,[2005.04.13\n답변 삭제\n화이티잉~~~ 얍!!]
38574,6632c1565bc2b816b6ca231a,사랑전선에 또 문제가...,이은희,2005.04.12,82,참 공감가네요!저두 두달전에...헤어졌답니다!첨 만날때부터 걱정은 됐지만...괜찮을...,[2005.04.13\n답변 삭제\n화이티잉~~~ 얍!!]
38575,6632c1575bc2b816b6ca231b,사랑전선에 또 문제가...,이은희,2005.04.12,82,참 공감가네요!저두 두달전에...헤어졌답니다!첨 만날때부터 걱정은 됐지만...괜찮을...,[2005.04.13\n답변 삭제\n화이티잉~~~ 얍!!]
38576,6632c1585bc2b816b6ca231c,사랑전선에 또 문제가...,이은희,2005.04.12,82,참 공감가네요!저두 두달전에...헤어졌답니다!첨 만날때부터 걱정은 됐지만...괜찮을...,[2005.04.13\n답변 삭제\n화이티잉~~~ 얍!!]


In [65]:
# 중복값 제거 후 확인
df_behcet = drop_duplicates(df_behcet, 'contents_text')
df_behcet['contents_text'].value_counts()

contents_text
참 공감가네요!저두 두달전에...헤어졌답니다!첨 만날때부터 걱정은 됐지만...괜찮을 줄 알았는데...만나고 얼마간은 아프지도 않고 좋았는데... 몇개월 되지 않아서 아프기 시작!병원에 들락 날락 거리고...지방에 내려와서 자주 보지도 못하고...그러다 보니 서로 힘들어지고...그쪽 어머니께서 그런 말씀을 하셨다네요!아픈 여자랑 살면 고생한다고 힘들다고...글구 본인도 주위에서 축복받는 결혼 하고싶고 아파하는 모습 옆에서 지켜보기 힘들 것 같다고...그당시는 수용하고 이해하고 내 자신이 한심도 했었지만... 아프다고 사랑하는 사람과 해어졌다고 인생이 끝난건 아니니깐... 밝고 건강하게 살려고 요즘 바쁘게...노력하고 있답니다! 님께서 어떤 결정을 내리실지...두분의 사랑이 얼마나 깊은지 전 알지 못하지만... 행복한 결말을 맺길 바라는 맘은 제 맘 가득이랍니다! 화이팅하세요!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      1
한참  몸이 않좋아 열심히 치료하다\n잠시 한숨을 돌립니다.\n\n요즘 집압 공원에 벗꽃이 반개해서\n산책하

In [66]:
# 각 컬럼별 전처리(제목)
df_behcet['contents_title']

0              어느봄날에...
1       세상에서 가장 좋은 이별이란
2                  사람살려
3                봄이 왔어요
4        신년맞이 떡국떡 감사의 글
             ...       
2436              배영반님께
2437         나는 이런 여자예요
2438        미안한 투정이죠?!!
2439     홈피 차라리 옛것이....
2440     사랑전선에 또 문제가...
Name: contents_title, Length: 2441, dtype: object

In [67]:
# 각 컬럼별 전처리(내용)
df_behcet.loc[:,'contents_text'] = df_behcet['contents_text'].apply(remove_text)
df_behcet

,_id,contents_title,contents_writer,contents_date,click_count,contents_text,reply_list
0,6631f76c5bc2b816b6c98c6c,어느봄날에...,배영반,2024.04.07,19,한참 몸이 않좋아 열심히 치료하다 잠시 한숨을 돌립니다. 요즘 집압 공원에 벗꽃...,"[이재강 2024.04.10\n답변\n오~~뚱뚱해 보여요,, 각도 비스듬이 해서 찍..."
1,6631f7835bc2b816b6c98c6d,세상에서 가장 좋은 이별이란,한현옥,2024.03.19,19,폐암말기로 투병하던 길똥씨.. 방사선을 머리에 쐬여 영혼마저 암세포와 함께 파쇄됐나...,[이재강 2024.03.19\n답변\n잘 읽었습니다.]
2,6631f7855bc2b816b6c98c6e,사람살려,한현옥,2024.03.01,22,00:00 00:00 옻순을 얼마나 좋아하는지 ...,[]
3,6631f7875bc2b816b6c98c6f,봄이 왔어요,한현옥,2024.02.27,16,봄에 심을 햇강낭콩을 종류대로 사려고 오일장만 서면 가서 눈여겨 보았었다. 작년.제...,[이재강 2024.02.27\n답변\n^^ 몸 생각하셔서 쉬엄쉬엄 농사 지으세요]
4,6631f7875bc2b816b6c98c70,신년맞이 떡국떡 감사의 글,이재강,2024.02.01,13,할리데이비슨 할리천사와 코리아챕터 감사드립니다~ 선정해주신 환우회 임원님들과 연합...,[]
...,...,...,...,...,...,...,...
2436,663225785bc2b816b6c9965c,배영반님께,김태용,2005.04.27,104,총무님 수술이 잘되셨다니 봄의 꽃향기와함께 기쁘게 생각합니다. 환우회 도움도안되는 ...,"[김태용 2005.04.27\n답변\n안녕하세요 심인보입니다, 오랜만이네요 그동안 ..."
2437,6632257c5bc2b816b6c9965d,나는 이런 여자예요,한현옥,2005.04.25,127,아침 출근길에 푸대자락을 들고 꽃들이 만개한 공원에 올라갔다. 웬 푸대자락이냐고? ...,[홍은표 2005.04.26\n답변\n꽃향기 폴폴 풍기는 꽃잎 염색하는 천연염색가님...
2438,6632257f5bc2b816b6c9965e,미안한 투정이죠?!!,임용미,2005.04.25,76,글을 읽다보니 저는 다른 환우들에 비해 덜 아픈거라는 생각이 듭니다.. 저는 구혈이...,[한현옥 2005.04.26\n답변 삭제\n듣다보니 미안한 투정이 아니고 당연한 투...
2439,663225835bc2b816b6c9965f,홈피 차라리 옛것이....,운영진,2005.04.25,75,새로이 만든 홈페이지이용에대한 개선점에 대한 의견 감사드립니다 비난홈과같은 많은 기...,[한현옥 2005.04.25\n답변 삭제\n다른건 다 몰라도 쪽지기능만은 살려주셨으...


In [68]:
behcet_list = behcet_series.tolist()
behcet_list

['어느봄날에... 한참  몸이 않좋아 열심히 치료하다 잠시 한숨을 돌립니다.  요즘 집압 공원에 벗꽃이 반개해서 산책하기 좋은 환경이 되었내요.  오늘 잠시산책을하러 갔더니... 어휴, 사람이 어찌나 많은지...   그래도 오랜만에 사람 구경도하고  꽃 구경도 하니 기분이 좋아 졌습니다.   이봄이 가기전에 들 얼굴보고 차도 한잔하고 해야 하는데. 아쉬움 이 가득 입니다.   모두틀 따뜻한 봄 기운가득채우셔서 조금은 덜 힘들고,,아파하셨으면 좋겠네요.         ',
 '세상에서 가장 좋은 이별이란 폐암말기로 투병하던 길똥씨.. 방사선을 머리에 쐬여 영혼마저 암세포와 함께 파쇄됐나보다. 저기 문지방에 서있는 흐연 할매가 누구간 가도 않고 나만츠다보냐고 내게 묻는데.  노망끼 작렬하여. 밤 낮 없이. 헛것과 희롱하더만..  거진 갈때가 가까웠던지 운신하기 힘겨워져 화장실에 부측하여 가다가 몸을 부려버린 그의 무게에.나도 함께 무너졌다. 내 몸도 몸이..아닌데.. 이러다간 병자도 간병하는 나도 바워내지 못할테니. 이때.  측근의 강력 추진으로..호스피스 병동으로 가기로 결정하였다. 폐암은 둘째치고  방사선 치매로 사지분간 못하는 길똥씨를 데려갈  호스피스 차가 마당으로 들어왔고 .길똥씨를  들것에 실어 나가려던 찰라.  문앞을 막고서. "안돼!.. 그렁게..말하자면..이집 나가면  죽으러 거쟎아? 그래요. 당신..죽어도 집에서 죽어.! 당신 영 떠날때 내가 못보쟎아? 내가 치다보는 내곁에서 죽어!"  결국 호스피스로 향해 들것에 실려 그야말로 나가죽으러  갈뻔 했던 길똥씨..  서른 세 밤  나랑  해로를 더 하다가.. 코스모스 환장하게 흐느적 거리는 가을볕 끝내주던 날에.. 세상과  작별인사 확실하게 하고서.  두 눈 빡뜨고 치다보는내 품 앞에서 숨을 거두었다. 영혼의 집.. 내 집에서... 가장 살기 좋은 곳이 가장 죽기도 좋은집 아라고. 시인 이생진님 시에 써 있더라..  백살이 코앞인 울어매  노인네 신수. 밤새 안녕인것을.. 자칫 건강에 빨간블이 켜

###### 답글 전처리 보류

###### guillainbarre

###### neurofibromatosis

###### rheumatism

##### 형태소 분석

In [74]:
# 모든 자연어 list 합치기
total_lists = KMSS_list + dcinside_list + behcet_list
total_lists

["늦었습니다. 어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원을 하셔서 집에 와서 여러가지 일을 하다보니 글 올리는 것이 생각이 났습니다.;오늘은 우리가 많이 먹는 포도에 대한 효능과 부작용에 대한 글입니다.포도 효능 11가지한여름이 지나가고 가을이 올 때쯤 되면 한참 달콤한 과일을 먹을 수 있어서 좋은 거 같다. 달콤한 복숭아도 좋고, 새콤한 자두도 좋지만 한 알 한 알 톡톡 터지는 식감이 좋은 포도도 너무 좋아해요. 아이들 간식부터 어르신들 입맛까지 사로잡은 과일로 누구나 좋아하는 음식으로 손꼽힌다.7~8월이 제철이어서 무더운 여름에 맛있게 먹을 수 있는 과일입니다. 특유의 향이 좋고 과즙이 풍부해서 과일 자체로도 맛있지만 주스로 마시거나 건포도로 가공해서 먹기도 한다. 우리나라는 이 과일을 꽤 옛날부터 먹어왔습니다.포도 효능1. 혈압을 조절해 준다.포도는 심장 건강에 도움이 되는 한 부분으로 혈압에 긍정적인 영향을 미칩니다. 혈압은 동맥혈관 벽에 대항한 혈액의 압력을 말한다. 이 혈액의 압력이 여러 원인으로 인해 높아진 상태를 고혈압이라 하는데, 우리나라 성인 인구의 30%가량이 고혈압 증상이 있는 것으로 추정하고 있습니다. 포도에는 칼륨과 섬유질이 풍부해서 나트륨의 배출을 돕습니다. 나트륨이 고혈압을 유발하는 원인 중 한 가지입니다. 포도를 적정량 꾸준히 섭취하다 보면 나트륨으로 인한 고혈압의 위험을 낮출 수 있습니다.2. 치매를 예방해 준다.포도 씨의 폴리페놀 성분은 알츠하이머의 진전을 늦춰줍니다. 연구에 의하면 실험용 생쥐에게 포도 씨의 폴리페놀 추출물을 5개월 동안 투여한 결과, 기억력 감퇴에 영향을 주는 ‘아밀로이드 베타-56’이라 하는 신경독소 물질이 감소한 것으로 나타났습니다.3. 항암 효과를 볼 수 있다.포도에는 폴리페놀이라는 항산화 성분이 포함되어 있습니다. 이들은 산화방지제 역할을 하여 암 관리와과 세포 손상 및 보호에 도움을 줄 수 있습니다. 특히 적포도에는 레스베라트롤이라는 폴리페놀의 일종인 성분이 많이 함유

###### 불용어 제거

In [ ]:
import re
patterns = '()'
series = []
for total_list in total_lists:
    total_list = str(total_list)
    text_regex = re.sub(pattern=patterns,repl='',string=total_list)
    if text_regex != '':
        series.append(text_regex)
series[:2]

In [ ]:
# 불용어 리스트 생성 (예시)
stopwords = ['자꾸','하루','신경섬유','지금','생각','이제','','갑자기','여기','사람','한번','오늘','그냥','희귀','가기','새끼','진짜' , '가끔','가든','가야','이번','제발','병신','시발','게이','dc','app','official','씨발','ㅆㅂ', '가', '고','좆', '을', '를', '이', '는']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 토크나이징 함수 정의
# def tokenizer(raw, pos=["Noun, Adverb, Verb"], stopword=stopwords):  # error ["Noun, Adverb, Verb"]
#     return [
#         word for word, tag in okt.pos(
#             raw, 
#             norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
#             stem=True    # stemming 바뀌나->바뀌다
#             )
#             if len(word) > 1 and tag in pos and word not in stopword
#     ]

def tokenizer(raw, pos=["Noun", "Adverb", "Verb"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
    ]

In [ ]:
# tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
# tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=2)
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.85, min_df=2)

In [ ]:
my_series = pd.Series(new_list)

In [ ]:
my_series

In [ ]:
new_list = []
for x in series:
    if x == '':
        pass
    else : 
        new_list.append(x)

In [ ]:
new_list

In [ ]:
# 여기서는 각 문서가 한글 문장으로 되어있으므로 그냥 결합해도 됩니다.
# 그러나 다른 언어의 경우에는 필요한 전처리를 수행해야 할 수도 있습니다.
cleaned_series = [doc.strip() for doc in series if doc.strip()]
corpus = ' '.join(cleaned_series)

# 필요한 경우 특수 문자 등을 제거하는 등의 추가적인 전처리를 수행합니다.
corpus = re.sub(r'[^\w\s]', '', corpus)

In [ ]:
features = tfidfVectorizer.fit_transform([corpus])
features.toarray()[:2]

In [ ]:
series = [' '.join(x.split()) for x in series]
features = tfidfVectorizer.fit_transform(series)
features.toarray()[:2]

###### LDA를 활용하여 Topic 추출

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
components_3  = LatentDirichletAllocation(n_components=3, n_jobs=-1) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기
components_3.fit(features)

In [ ]:
vis = pyLDAvis.lda_model.prepare(components_3, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소